In [1]:
library("TSP")
library("hash")
library("sets")

dataTSP <- read_TSPLIB("/home/LC/mailo01/TSP-ML-Approach/src/R/CLUSTER_DIV/main.tsp")


pointSet <- hash()
noPts <- length(dataTSP)/2
for (i in 1:noPts){
    pointSet[[toString(i)]] <- tuple(dataTSP[i], dataTSP[i+noPts])
}

dist_pts <- function(pt1,pt2){
        return(sqrt((pt1[[1]]-pt2[[1]])^2 + (pt1[[2]]-pt2[[2]])^2))
}

angle <- function(x,y){
  dot.prod <- x%*%y 
  norm.x <- norm(x,type="2")
  norm.y <- norm(y,type="2")
  theta <- acos(dot.prod / (norm.x * norm.y))
  as.numeric(theta)
}

angle_pts <- function(pt1,pt2){
    
    if (pt1[[2]] -pt2[[2]] == 0){
        if (pt1[[1]] -pt2[[1]]>0){
            return(pi/2)
        }
        if (pt1[[1]] -pt2[[1]]<0){
            return(-pi/2)
        }
        if (pt1[[1]] -pt2[[1]]==0){
            return(0)
        }
    } else{
        x<- pt1[[1]] -pt2[[1]]
        y <-pt1[[2]] -pt2[[2]]
        angle <- atan((y)/(x))
        if (x < 0 & y>0){
            angle <- pi + angle
        }
        if (x >0 & y <0){
            angle <-2*pi +angle
        }
        if (x < 0 & y<0){
            angle <- pi + angle
        }
        return(angle)
        }
}


hash-2.2.6.1 provided by Decision Patterns


Warning message in file(con, "r"):
“cannot open file '/home/LC/mailo01/TSP-ML-Approach/src/R/CLUSTER_DIV/main.tsp': No such file or directory”


ERROR: Error in file(con, "r"): cannot open the connection


In [2]:
radius <-7
plot(dataTSP,xlim=c(0,100), ylim=c(0,100), pch=20)
symbols(dataTSP,circles=rep(radius,noPts), add=T, inches=F)

##manual pick
manual_point1 <- values(pointSet, keys =1)
print(manual_point1)
#points(manual_point1[[1]],manual_point1[[2]], col="2")

manual_point2<- values(pointSet, keys =1)
print(manual_point2)
points(manual_point2[[1]],manual_point2[[2]], col="3")

angle_pts(manual_point2,manual_point1)

ERROR: Error in plot(dataTSP, xlim = c(0, 100), ylim = c(0, 100), pch = 20): object 'dataTSP' not found


In [3]:
pivot_point <- values(pointSet, keys =1)

angle_based <- list()
distance_based <-list()

for (i in 1:noPts){
    compare_point <- values(pointSet, keys= i)
    element <-list(list(i, dist_pts(pivot_point,compare_point), angle_pts(compare_point,pivot_point)))
    angle_based <- append(angle_based, element)
    distance_based <- append(distance_based, element)
}




ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'x' in selecting a method for function 'values': object 'pointSet' not found


In [6]:
start_time <- Sys.time()

angle_based <- angle_based[order(sapply(distance_based, '[[', 3))]
end_time <- Sys.time()
end_time - start_time

ERROR: Error in eval(expr, envir, enclos): object 'angle_based' not found


In [7]:
start_time <- Sys.time()

distance_based <- distance_based[order(sapply(distance_based,'[[',2))]
end_time <- Sys.time()
end_time - start_time

ERROR: Error in eval(expr, envir, enclos): object 'distance_based' not found


In [4]:
get_item <- function(item_to_get, database){
    counter <-1
    cont <- TRUE 
    while (counter <= length(database) & cont == TRUE){
        data_pt <-database[[counter]]
        if (data_pt[1] == item_to_get){
            return(data_pt)
            cont <- FALSE
        }
        counter <- counter +1
    }
}

get_pts_range_dist <- function(lower_bound, higher_bound){
    cont = TRUE
    counter <-1
    eligible_vertices <-list()
    while (counter <= length(distance_based) & cont ==TRUE){
        distance_wrt_pivot <- distance_based[[counter]]
        
        if (distance_wrt_pivot[[2]] >= lower_bound & distance_wrt_pivot[[2]]<= higher_bound){
            eligible_vertices <- append(eligible_vertices, distance_wrt_pivot[[1]])
        }
        if (distance_wrt_pivot[[2]]> higher_bound){
            cont <- FALSE
        }
        counter <- counter +1
    }
    return(eligible_vertices)
}

obtain_angle_range <- function(lower_bound, higher_bound){
    cont = TRUE
    counter <-1
    eligible_vertices <-list()
    while (counter <= length(angle_based) & cont ==TRUE){
        angle_wrt_hor <- angle_based[[counter]]
        
        if (angle_wrt_hor[[3]] >= lower_bound & angle_wrt_hor[[3]]<= higher_bound){
            eligible_vertices <- append(eligible_vertices, angle_wrt_hor[[1]])
        }
        if (angle_wrt_hor[[3]]> higher_bound){
            cont <- FALSE
        }
        counter <- counter +1
    }
    return(eligible_vertices)
}

conform_angle <- function(angle){
    if (angle <0){
        angle <- 2*pi + angle
    }
    if (angle >2*pi){
        angle <- angle - 2*pi
    }
    
    return(angle)
}

get_pts_range_angle <- function(distance_from_pivot, radius, angle){
     eligible_vertices <-list()
    if (distance_from_pivot < radius){
        eligible_vertices <- append(eligible_vertices, angle_based)
    }else{
        if (radius == distance_from_pivot){
            #r = d, gives the angle range that is 180 degrees. Gives which side?
            higher_bound <- conform_angle(pi/2 - (pi - angle))
            lower_bound<- conform_angle(higher_bound + pi)
            eligible_vertices <- append(eligible_vertices, obtain_angle_range(lower_bound, higher_bound))
        }else{
            #r > d
            phi <- asin(2*radius/distance_from_pivot)
            if (angle + phi > 2*pi){
                diff <-  angle + phi -2*pi 
                eligible_vertices <- append(eligible_vertices, obtain_angle_range(0,diff))
                eligible_vertices <- append(eligible_vertices, obtain_angle_range(angle-phi, 2*pi))
            } else {
                if (angle - phi < 0){
                    diff <- angle - phi 
                    eligible_vertices <- append(eligible_vertices, obtain_angle_range(2*pi + diff, 2*pi))
                    eligible_vertices <- append(eligible_vertices, obtain_angle_range(0, angle+phi))
                }else{
                    eligible_vertices <- append(eligible_vertices, obtain_angle_range(angle-phi, angle+phi))
                }
            }
        }
    }    
    return(eligible_vertices)

}

In [8]:
start_time <- Sys.time()
current_tree <- list()
checklist <-(c(1:noPts))
while (length(checklist) != 0){

    branch <- list()
    
    old_branch_len<- length(branch)

    startPt = checklist[1]
    branch<-append(branch, startPt)

    counter =1

    while(old_branch_len != length(branch)){

        current_branch_len <- length(branch)
        
        start <-old_branch_len+1
        pts_to_connect <- c(branch[start:current_branch_len])


        checklist <-checklist[!checklist %in% pts_to_connect]

        old_branch_len= current_branch_len

        for (connect_pt in pts_to_connect){
            print("point")
            print(connect_pt)
            ## to yield an estimates of near by points 
            pivot_data <- get_item(connect_pt, distance_based)
            dist_wrt_pivot <- pivot_data[[2]][1]
            angle_wrt_hor <-pivot_data[[3]][1]
            if (dist_wrt_pivot == 0){
                possible_connect <- get_pts_range_dist(0,2*radius)
                
            } else {
                possible_angle <- get_pts_range_angle(dist_wrt_pivot, radius, angle_wrt_hor)
                
                possible_dist <- get_pts_range_dist(dist_wrt_pivot-2*radius, dist_wrt_pivot+2*radius)
                possible_connect <- intersect(possible_angle, possible_dist)
            }
            print("possible pt to connect")
            print(possible_connect)
            for (comparePt in possible_connect){

                if (!comparePt %in%branch){
                    vPt1 <-values(pointSet, keys=connect_pt) 
                    vPt2 <-values(pointSet, keys=comparePt) 
                    
                    if (dist_pts(vPt1,vPt2) <= 2*radius){
                        
                        branch<-append(branch,comparePt)
                        
                        checklist <-checklist[!checklist %in% c(comparePt)]

                        
                    } else {}
                } else{}
            }
        }
        counter <- counter +1
        
    }
    current_tree <- append(current_tree, list(branch))
}
end_time <- Sys.time()
end_time - start_time


ERROR: Error in eval(expr, envir, enclos): object 'noPts' not found


In [9]:
plot(dataTSP,xlim=c(0,100), ylim=c(0,100), pch=20)
symbols(dataTSP,circles=rep(radius,noPts), add=T, inches=F)

branchNo = 10
for (all in current_tree[[branchNo]]){
    pt <- values(pointSet, keys =all)
    points(pt[[1]],pt[[2]], col="3", pch=2)
}

manual_pt <- values(pointSet, keys =1)
#points(manual_pt[[1]],manual_pt[[2]], col="2", pch=5)
check <- c(22,29)
for (pt in check){
    #manual_pt <- values(pointSet, keys =pt)
    #points(manual_pt[[1]],manual_pt[[2]], col="4", pch=3)
}

vPt1 <-values(pointSet, keys=1) 
vPt2 <-values(pointSet, keys=29) 
vPt3 <-values(pointSet, keys =22)
dist_pts(vPt1,vPt2)

#vPt1
#vPt2
#vPt3
#angle_pts(vPt2,vPt1)

ERROR: Error in plot(dataTSP, xlim = c(0, 100), ylim = c(0, 100), pch = 20): object 'dataTSP' not found


In [10]:
for (branchNo in 1:length(current_tree)){
    pts  <- data.frame(x=c(), y=c())
    if ( length(current_tree[[branchNo]]) <=2){
        ##write solution to be the same as the output to .sol file
        file_name <- sprintf("/home/LC/mailo01/TSP-ML-Approach/src/R/CLUSTER_DIV/branch_%s_case_%s.sol",branchNo,1)
        if (length(branch) == 1){
            write(c(1,0), file_name)
        }

        if (length(branch) == 2){
            write(c(2, 1, 0), file_name )
        }

    }else{

        for (pt in branch){
            pt_data <- get_item(pt, distance_based)
            pts <- rbind(pts, c(pt_data[[2]],pt_data[3]))

        }
        etsp <- ETSP(pts)
            file_name <- sprintf("branch_%s_case_%s.tsp",branchNo,1)
            write_TSPLIB(etsp, file= file_name)
    }
}

ERROR: Error in current_tree[[branchNo]]: subscript out of bounds


In [11]:
## import library
library("TSP")


## listing of solutions provided by concorde
dataTSP <- read_TSPLIB("/home/LC/mailo01/TSP-ML-Approach/src/R/CLUSTER_DIV/main.tsp")
dataTSPsub <- read_TSPLIB("/home/LC/mailo01/TSP-ML-Approach/src/R/CLUSTER_DIV/branch_35_case_1.tsp")
all_solutions <- scan("/home/LC/mailo01/TSP-ML-Approach/src/R/CLUSTER_DIV/solution_files.txt", what=character())


Warning message in file(con, "r"):
“cannot open file '/home/LC/mailo01/TSP-ML-Approach/src/R/CLUSTER_DIV/main.tsp': No such file or directory”


ERROR: Error in file(con, "r"): cannot open the connection


In [12]:
convert_sol_format <- function(content){
    content <- content[-1]
    for (index in 1:length(content)){
        content[[index]] <- content[[index]] + 1
    }
    return(as.integer(content))
}

In [13]:
all_solutions[[50]]
content <- scan(all_solutions[[50]])

content
tour_info <- convert_sol_format(content)
tour_info


ERROR: Error in eval(expr, envir, enclos): object 'all_solutions' not found


In [14]:
tour <- as.TOUR(tour_info)
tour

ERROR: Error in as.TOUR(tour_info): object 'tour_info' not found


In [15]:
plot(dataTSPsub, tour)

ERROR: Error in plot(dataTSPsub, tour): object 'dataTSPsub' not found
